In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


# Cross entropy example
import numpy as np
# One hot
# 0: 1 0 0
# 1: 0 1 0
# 2: 0 0 1
Y = np.array([1, 0, 0])
 
Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])
print("loss1 = ", np.sum(-Y * np.log(Y_pred1)))
print("loss2 = ", np.sum(-Y * np.log(Y_pred2)))


######## 그러나 파이토치에는 함수로 제공한다. 
# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot

## 클래스인데, one hot 안해줘두 됨.
Y = Variable(torch.LongTensor([0]), requires_grad=False)

# input is of size nBatch x nClasses = 1 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 = ", l1.data, "\nPyTorch Loss2=", l2.data)

print("Y_pred1=", torch.max(Y_pred1.data, 1)[1])
print("Y_pred2=", torch.max(Y_pred2.data, 1)[1])

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)


## 맞은거니까 값이 낮게 나옴
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                 [1.1, 0.1, 0.2],
                                 [0.2, 2.1, 0.1]]))

## 틀린거니까 값이 크게나옴
Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                 [0.2, 0.3, 0.5],
                                 [0.2, 0.2, 0.5]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("Batch Loss1 = ", l1.data, "\nBatch Loss2=", l2.data)

('loss1 = ', 0.35667494393873245)
('loss2 = ', 2.3025850929940455)
('PyTorch Loss1 = ', 
 0.4170
[torch.FloatTensor of size 1]
, '\nPyTorch Loss2=', 
 1.8406
[torch.FloatTensor of size 1]
)
('Y_pred1=', 
 0
[torch.LongTensor of size 1]
)
('Y_pred2=', 
 1
[torch.LongTensor of size 1]
)
('Batch Loss1 = ', 
 0.4966
[torch.FloatTensor of size 1]
, '\nBatch Loss2=', 
 1.2389
[torch.FloatTensor of size 1]
)


## mnist 적용

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        
        ## pred이랑, target이랑 일치하면. 1 뱉는듯?
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313220
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.306010
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.296495
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.299145
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.296978
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.303520
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.303169
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.305560
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.304659
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.313391
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.307547
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.306327
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.294539
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.301056
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.302012
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.303540
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.304664
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.302392
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.299980
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.294625
Train Epoch: 1 [12800/60000 (

## 연습

In [16]:
def test():
    model.eval()
    test_loss=0
    correct = 0
    for data,target in test_loader:
        data, target = Variable(data), Variable(target)
        output = model(data)
        
        test_loss += criterion(output, target).data[0]
        
        # loss 값은 0.058
        
        
        pred = output.data.max(1,keepdim=True)[1]
        
        # 가장큰 값은 7

        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
        ## 맞춘개수를 correct에 계속 sum
        
    ## test_loss 다 더한것.
    
    test_loss/=len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

test()



Test set: Average loss: 0.0016, Accuracy: 9707/10000 (97%)



10000